In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
DATADIR = r"Directory Path"

CATEGORIES = ["jeans", "saree", "trouser"]

for category in CATEGORIES:
    path = os.path.join(DATADIR,category)  # create path to jeans, saree and trouser
    for img in os.listdir(path):  # iterate over each image per jeans, saree, trouser
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray') #graph it
        plt.show()  # display!

        break
    break

In [ ]:
import pathlib
data_dir = pathlib.Path(DATADIR)
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
jeans = list(data_dir.glob('jeans/*'))
jeans[:5]

In [ ]:
PIL.Image.open(str(jeans[1]))

In [ ]:
saree = list(data_dir.glob('saree/*'))
PIL.Image.open(str(saree[0]))

In [ ]:
trouser = list(data_dir.glob('trouser/*'))
PIL.Image.open(str(trouser[0]))

### Read dataset images from disk into numpy array using opencv

In [ ]:
clothes_images_dict = {
    'jeans': list(data_dir.glob('jeans/*')),
    'saree': list(data_dir.glob('saree/*')),
    'trouser': list(data_dir.glob('trouser/*')),
}

In [ ]:
clothes_labels_dict = {
    'jeans': 0,
    'saree': 1,
    'trouser': 2,
}

In [ ]:
clothes_images_dict['jeans'][:5]

In [ ]:
str(clothes_images_dict['jeans'][0])

In [ ]:
img = cv2.imread(str(clothes_images_dict['jeans'][0]))

In [ ]:
img.shape

In [ ]:
cv2.resize(img,(180,180)).shape

In [ ]:
X, y = [], []

for clothe_name, images in clothes_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(clothes_labels_dict[clothe_name])

In [ ]:
X = np.array(X)
y = np.array(y)

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Preprocessing: scale images

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

### Build convolutional neural network and train it

In [ ]:
num_classes = 3

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)

In [ ]:
model.evaluate(X_test_scaled,y_test)

### Here we see that while train accuracy is very high (99%), the test accuracy is significantly low (88.67%) indicating overfitting. Let's make some predictions before we use data augmentation to address overfitting

In [ ]:
predictions = model.predict(X_test_scaled)
predictions

In [ ]:
score = tf.nn.softmax(predictions[0])

In [ ]:
np.argmax(score)

In [ ]:
y_test[0]

### Improve Test Accuracy Using Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

### Original Image

In [ ]:
plt.axis('off')
plt.imshow(X[0])

In [ ]:
data_augmentation(X)[0]

### Newly generated training sample using data augmentation

In [ ]:
plt.axis('off')
plt.imshow(data_augmentation(X)[0].numpy().astype("uint8"))

### Train the model using data augmentation and a drop out layer

In [ ]:
num_classes = 3

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
pre=model.predict(X_test_scaled)
pre

In [ ]:
y_pred_classes = [np.argmax(element) for element in pre]
y_pred_classes

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print('classification_report\n',classification_report(y_test,y_pred_classes))

In [ ]:
#plot confusion matrix heatmap
import seaborn as sns
conf_mat = confusion_matrix(y_test,y_pred_classes)

ax=plt.subplot()

sns.heatmap(conf_mat,annot=True,ax=ax,linewidth=5,linecolor='r',center=0)

ax.set_xlabel('Predicted Labels');ax.set_ylabel('True Labels')

ax.set_title('Confusion matrix')

plt.show()